In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import random
from scipy.stats import mode

### 데이터 내려받기

In [3]:
train_df = pd.read_csv("../Data/predict_future_sales_data/sales_train.csv")
test_df = pd.read_csv("../Data/predict_future_sales_data/test.csv")

submission = pd.read_csv("../Data/predict_future_sales_data/sample_submission.csv")

item_categories_df = pd.read_csv("../Data/predict_future_sales_data/item_categories.csv")
items_df = pd.read_csv("../Data/predict_future_sales_data/items.csv")
shops_df = pd.read_csv("../Data/predict_future_sales_data/shops.csv")


### 데이터 탐색

In [4]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
train_df.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [7]:
np.sort(train_df["shop_id"].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

In [8]:
train_df["date_block_num"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [9]:
item_categories_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [10]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [11]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [12]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [13]:
train_df[train_df['item_id']==22167]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
849867,29.09.2013,8,43,22167,299.00,1.0
852087,28.09.2013,8,42,22167,299.00,1.0
852088,30.09.2013,8,42,22167,299.00,5.0
857451,30.09.2013,8,22,22167,299.00,1.0
857452,29.09.2013,8,22,22167,299.00,1.0
...,...,...,...,...,...,...
2930883,16.10.2015,33,22,22167,299.00,2.0
2930884,17.10.2015,33,22,22167,254.15,1.0
2930885,20.10.2015,33,22,22167,299.00,1.0
2930886,23.10.2015,33,22,22167,299.00,1.0


In [14]:
train_df[["item_id", "item_price"]].groupby('item_id').var()


,item_price
item_id,
0,NaN
1,0.000000
2,0.000000
3,882.000000
4,NaN
...,...
22165,0.000000
22166,0.000000
22167,160.713988


## 데이터 처리

In [15]:
# 학습데이터 date 제거
train_df = train_df.drop(['date'], axis=1)
train_df

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1.0
1,0,25,2552,899.00,1.0
2,0,25,2552,899.00,-1.0
3,0,25,2554,1709.05,1.0
4,0,25,2555,1099.00,1.0
...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0
2935845,33,25,7460,299.00,1.0
2935846,33,25,7459,349.00,1.0
2935847,33,25,7440,299.00,1.0


In [16]:
# 학습데이터에 item_category_id 추가
train_df =  pd.merge (train_df, items_df[['item_id', 'item_category_id']], how='left', on='item_id')

train_df

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,0,59,22154,999.00,1.0,37
1,0,25,2552,899.00,1.0,58
2,0,25,2552,899.00,-1.0,58
3,0,25,2554,1709.05,1.0,58
4,0,25,2555,1099.00,1.0,56
...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,55
2935845,33,25,7460,299.00,1.0,55
2935846,33,25,7459,349.00,1.0,55
2935847,33,25,7440,299.00,1.0,57


In [17]:
test_df.size

642600

In [18]:
# date_block_num, item_price, item_category_id 추가 
test_df['date_block_num'] = 34
item_id_price = train_df[['item_id', 'item_price']].groupby('item_id').agg(pd.Series.mode).reset_index()

item_id_price['item_price'] =  item_id_price['item_price'].apply(lambda x: float(x) if isinstance(x, (int, float)) else float(x[-1]))

item_id_price = item_id_price.astype(float)
test_df = pd.merge(test_df, item_id_price, how='left', on='item_id')
test_df = pd.merge(test_df, items_df[['item_id', 'item_category_id']], how='left',on='item_id')
test_df

,ID,shop_id,item_id,date_block_num,item_price,item_category_id
0,0,5,5037,34,2599.0,19
1,1,5,5320,34,NaN,55
2,2,5,5233,34,599.0,19
3,3,5,5232,34,599.0,23
4,4,5,5268,34,NaN,20
...,...,...,...,...,...,...
214195,214195,45,18454,34,199.0,55
214196,214196,45,16188,34,1223.0,64
214197,214197,45,15757,34,199.0,55
214198,214198,45,19648,34,99.0,40


## 학습 1

In [19]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 

import lightgbm as lgb
import optuna

folds = 3
kf = KFold(n_splits=folds)



train_x = train_df.drop(["item_cnt_day"], axis=1)
train_y = train_df['item_cnt_day']


### 하이퍼파라미터 최적화

In [20]:


def objective(trial):
    rmses = []
    params = {
        "objective":"regression",
        "random_seed":1234,
        "learnign_rate":0.05,
        "n_estimator":1000,
        "num_leaves":trial.suggest_int("num_leaves", 4, 64),
        "max_bin":trial.suggest_int("max_bin", 50, 200),
        "bagging_fraction":trial.suggest_uniform("bagging_fraction", 0.4, 0.9),
        "bagging_freq":trial.suggest_int("bagging_freq", 1, 10),
        "feature_fraction":trial.suggest_uniform("feature_fraction", 0.4, 0.9),
        "min_data_in_leaf":trial.suggest_int("min_data_in_leaf", 2, 16),
        "min_sum_hessian_in_leaf":trial.suggest_int("min_sum_hessian_in_leaf", 1, 10)
    }
    for train_index, val_index in kf.split(train_x):
        x_train = train_x.iloc[train_index]
        x_valid = train_x.iloc[val_index]
        y_train = train_y.iloc[train_index]
        y_valid = train_y.iloc[val_index]
        
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

        model_lgb = lgb.train(params,
                                lgb_train,
                                valid_sets=lgb_eval,
                                num_boost_round=100,
                                early_stopping_rounds=20,
                                verbose_eval=10
                            )
        y_pred = model_lgb.predict(x_valid, num_iteration=model_lgb.best_iteration)
        tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
        rmses.append(tmp_rmse)
        
    score = sum(rmses)/len(rmses)
    
    return score

In [21]:
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
# study.optimize(objective, n_trials=100)
# study.best_params

In [22]:
lgbm_params = {
        "objective":"regression",
        "random_seed":1234,
        "learnign_rate":0.05,
        "n_estimator":1000,
        'num_leaves': 25,
        'max_bin': 194,
        'bagging_fraction': 0.8157596999360686,
        'bagging_freq': 8,
        'feature_fraction': 0.6451526732743686,
        'min_data_in_leaf': 3,
        'min_sum_hessian_in_leaf': 6
    }

In [23]:
models = []
rmses = []

for train_index, val_index in kf.split(train_x):
    x_train = train_x.iloc[train_index]
    x_valid = train_x.iloc[val_index]
    y_train = train_y.iloc[train_index]
    y_valid = train_y.iloc[val_index]
    
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
    
    model_lgb = lgb.train(lgbm_params,
                          lgb_train, 
                          valid_sets=lgb_eval,
                          num_boost_round=100,
                          early_stopping_rounds=20,
                          verbose_eval=10,
                          )
    y_pred = model_lgb.predict(x_valid, num_iteration=model_lgb.best_iteration)
    tmp_rmse = np.sqrt(mean_squared_error(round(y_valid), y_pred))
    print(tmp_rmse)
    
    models.append(model_lgb)
    rmses.append(tmp_rmse)
    

[LightGBM] [Warning] Unknown parameter: learnign_rate
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Unknown parameter: learnign_rate
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Info] Total Bins 542
[LightGBM] [Info] Number of data: 1957232, number of used features: 5
[LightGBM] [Warning] Unknown parameter: learnign_rate
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Info] Start training from score 1.260286
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l2: 2.5115
[20]	valid_0's l2: 2.45256
Early stopping, best iteration is:
[3]	valid_0's l2: 2.19782
1.4825054635082833
[LightGBM] [Warning] Unknown parameter: learnign_rate
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Unknown parameter: learnign_rate
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Info] Total Bins 546
[LightGBM] [Info] Number of data: 1957233, number of used features: 5
[LightGBM] [Warni

In [24]:
print(sum(rmses)/len(rmses))

2.3473088414692813


In [25]:
test_df

,ID,shop_id,item_id,date_block_num,item_price,item_category_id
0,0,5,5037,34,2599.0,19
1,1,5,5320,34,NaN,55
2,2,5,5233,34,599.0,19
3,3,5,5232,34,599.0,23
4,4,5,5268,34,NaN,20
...,...,...,...,...,...,...
214195,214195,45,18454,34,199.0,55
214196,214196,45,16188,34,1223.0,64
214197,214197,45,15757,34,199.0,55
214198,214198,45,19648,34,99.0,40


In [26]:
test_x = test_df.drop(['ID'], axis=1)

preds  = []

for model in models:
    pred = model.predict(test_x)
    preds.append(pred)




In [27]:
models[0].feature_importance()

array([20, 10, 16, 13, 13], dtype=int32)

In [28]:
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

In [29]:
len(preds_mean)
preds_mean

array([1.29649264, 1.36562556, 1.1794869 , ..., 1.11200318, 1.11456925,
       1.13699365])

In [30]:
submission["item_cnt_month"] = preds_mean

In [31]:
#submission.to_csv("../submit/predict_futere_sales_submit01.csv", index=False)

## 학습1 xgboost

In [32]:
import xgboost as xgb

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [33]:
def objective(trial):
    rmses = []
    lr_list =[0.0001, 0.0005,  0.001, 0.005, 0.01, 0.05, 0.1]
    xgb_params = {
        "learning_rate":trial.suggest_categorical("learning_rate", lr_list),
        "seed":1234,
        "max_depth":trial.suggest_int("max_depth",  3, 16),
        "colsample_bytree":trial.suggest_uniform("colsample_bytree", 0.2, 0.9),
        "sublsample":trial.suggest_uniform("sublsample", 0.2, 0.9),
    }
    for train_index, val_index in kf.split(train_x):
        x_train = train_x.iloc[train_index]
        x_valid = train_x.iloc[val_index]
        y_train = train_y.iloc[train_index]
        y_valid = train_y.iloc[val_index]
        
        xgb_train = xgb.DMatrix(x_train, label=y_train)
        xgb_eval = xgb.DMatrix(x_valid, label=y_valid)
        evals = [(xgb_train, "train"), (xgb_eval, "eval")]
        model_xgb = xgb.train(xgb_params,
                            xgb_train,
                            evals=evals,
                            num_boost_round=100,
                            early_stopping_rounds=20,
                            verbose_eval=10,
                            )
        y_pred = model_xgb.predict(xgb_eval)
        tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
        rmses.append(tmp_rmse)
        
    score = sum(rmses)/len(rmses)
    
    return score

In [34]:
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler())
# study.optimize(objective, n_trials=100)
# study.best_params

{'learning_rate': 0.1,
 'max_depth': 7,
 'colsample_bytree': 0.8189686230514499,
 'sublsample': 0.47799459991435284}

In [35]:
xgb_params = {
        "learning_rate":0.1,
        "seed":1234,
        "max_depth":7,
        "colsample_bytree":0.8189686230514499,
        "sublsample":0.47799459991435284,
    }

In [56]:
models_xgb =[]
rmses_xgb = []


for train_index, val_index in kf.split(train_x):
    x_train = train_x.iloc[train_index]
    x_valid = train_x.iloc[val_index]
    y_train = train_y.iloc[train_index]
    y_valid = train_y.iloc[val_index]

    model = xgb.XGBRegressor(max_depth=7, n_estimators=1000, colsample_bytree=0.8189686230514499, sublsample=0.47799459991435284, tree_method='gpu_hist', gpu_id=0)
    model.fit(x_train, y_train, eval_metric='rmse', eval_set=[(x_valid, y_valid)],verbose=10, early_stopping_rounds=20)
    
    y_pred = model.predict(x_valid)
    tmp_rmse = np.sqrt(mean_squared_error(round(y_valid), y_pred))
    print(tmp_rmse)
    
    models_xgb.append(model)
    rmses_xgb.append(tmp_rmse)

[02:29:34] WARNING: /workspace/src/learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:1.57628
Will train until validation_0-rmse hasn't improved in 20 rounds.
[10]	validation_0-rmse:1.46831
[20]	validation_0-rmse:1.48327
Stopping. Best iteration:
[5]	validation_0-rmse:1.46081

1.46080684012204
[02:29:35] WARNING: /workspace/src/learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:1.95068
Will train until validation_0-rmse hasn't improved in 20 rounds

In [57]:
# models_xgb =[]
# rmses_xgb = []




# for train_index, val_index in kf.split(train_x):
#     x_train = train_x.iloc[train_index]
#     x_valid = train_x.iloc[val_index]
#     y_train = train_y.iloc[train_index]
#     y_valid = train_y.iloc[val_index]

    
#     xgb_train = xgb.DMatrix(x_train, label=y_train)
#     xgb_eval = xgb.DMatrix(x_valid, label=y_valid)
#     evals = [(xgb_train, "train"), (xgb_eval, "eval")]
    
#     model_xgb = xgb.train(xgb_params,
#                           xgb_train, 
#                           evals=evals,
#                           num_boost_round=100,
#                           early_stopping_rounds=20,
#                           verbose_eval=10,
#                           )
#     y_pred = model_xgb.predict(xgb_eval)
#     tmp_rmse = np.sqrt(mean_squared_error(round(y_valid), y_pred))
#     print(tmp_rmse)
    
#     models_xgb.append(model_xgb)
#     rmses_xgb.append(tmp_rmse)
    

In [53]:
print(sum(rmses)/len(rmses))

2.3473088414692813


In [54]:
test_x_pp = test_df.drop(['ID'], axis=1)
item_price_mean = test_df[['item_price']].mean()
test_x_pp.loc[(test_x['item_price'].isnull()),'item_price'] = item_price_mean[0]
test_x_pp = test_x_pp[['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_category_id']]
xgb_test= xgb.DMatrix(test_x_pp)

xgb_preds = []

for model in models_xgb:
    pred = model.predict(xgb_test)
    preds.append(pred)

xgb_preds = np.array(preds)
xgb_preds = np.mean(preds, axis=0)
xgb_preds

TypeError: can not initialize DMatrix from DMatrix

In [39]:
submission['item_cnt_month'] = xgb_preds

In [40]:
#submission.to_csv("../submit/predict_futere_sales_submit02.csv", index=False)

In [41]:
len(preds_mean)
np.round(xgb_preds)

array([1., 1., 1., ..., 1., 1., 1.])

In [42]:
preds_ans = preds_mean * 0.5 + xgb_preds * 0.5
submission['item_cnt_month'] = preds_ans
#submission.to_csv("../submit/predict_futere_sales_submit03.csv", index=False)

In [43]:
submission['item_cnt_month'] = np.round(xgb_preds)
#submission.to_csv("../submit/predict_futere_sales_submit04.csv", index=False)

In [44]:
#sales_train = pd.read_csv('../Data/predict_future_sales_data/sales_train.csv')
# sales_train

NameError: name 'sales_train' is not defined